# Feature Engineering – RockSafeAI

This notebook fuses seismic, acoustic emission, and excavation datasets  
into a unified zone-wise time-series dataset for rockfall risk prediction.

## 1. Imports & Configuration

In [2]:
import numpy as np
import pandas as pd
from scipy.signal import welch
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler

## 2. Load Seismic Dataset

In [3]:
seismic_path = "../data/raw/hudsonmt.out"

seismic_signal = np.loadtxt(seismic_path)
print("Seismic samples shape:", seismic_signal.shape)

Seismic samples shape: (800000, 11)


## 3. Extract Seismic Features

In [4]:
def extract_seismic_features(signal, fs=100):
    vibration_energy = np.sqrt(np.mean(signal ** 2))
    
    freqs, psd = welch(signal, fs)
    psd_norm = psd / np.sum(psd)
    vibration_entropy = entropy(psd_norm)
    
    return vibration_energy, vibration_entropy


vib_energy, vib_entropy = extract_seismic_features(seismic_signal)

seismic_df = pd.DataFrame({
    "vibration_energy": [vib_energy],
    "vibration_entropy": [vib_entropy]
})

seismic_df

/var/folders/2c/v9px2hv14cd81r_d2kn8rbwm0000gn/T/ipykernel_4197/4249333842.py:4: UserWarning: nperseg=256 is greater than signal length max(len(x), len(y)) = 11, using nperseg = 11
  freqs, psd = welch(signal, fs)


,vibration_energy,vibration_entropy
0,1266.302159,"[12.622751715188262, 12.816217014764439, 12.89..."


## 4. Load Acoustic Emission Dataset

In [5]:
ae_path = "../data/raw/acoustic/AE_Damage_Detection_Dataset.csv"
ae_df = pd.read_csv(ae_path)

ae_df.head()

,Peak_Amplitude_dB,Energy_Release_J,Signal_Duration_ms,Rise_Time_ms,Frequency_Spectrum_Hz,Load_Condition,Impact_Energy_J,Temperature_C,Humidity_%,Fiber_Type,Resin_Type,Thickness_mm,Damage_Type
0,89.001761,3.580577,1.041566,1.680999,419.369234,High,19.121970,37.809478,20.138428,Carbon,Epoxy,2.339700,0
1,76.735892,2.485476,1.980918,0.259623,295.130227,Low,12.344785,28.407899,56.077152,Carbon,Polyester,6.104654,1
2,59.093434,5.046810,4.847871,0.935139,433.341953,Low,48.534111,31.626721,50.827115,Glass,Polyester,6.123763,3
3,66.466135,9.017995,3.275014,0.665465,228.025910,Low,10.831400,39.944994,27.861538,Glass,Polyester,8.024866,0
4,74.521489,7.272187,4.928054,1.694996,119.032971,Medium,7.761118,17.698709,67.247827,Glass,Epoxy,2.528944,3


## 5. Extract Acoustic Features

In [6]:
acoustic_energy = ae_df.select_dtypes(include=np.number).mean(axis=1)
acoustic_hit_rate = ae_df.select_dtypes(include=np.number).count(axis=1)

ae_features = pd.DataFrame({
    "acoustic_energy": acoustic_energy,
    "acoustic_hit_rate": acoustic_hit_rate
})

ae_features = ae_features.head(1)
ae_features

,acoustic_energy,acoustic_hit_rate
0,59.408371,10


## 6. Load Excavation Risk Dataset

In [7]:
exc_path = "../data/raw/excavation/excavation_risk_dataset.csv"
exc_df = pd.read_csv(exc_path)

exc_df.head()

,Soil_Type,Soil_Moisture_%,Shear_Strength_kPa,Bearing_Capacity_kPa,Excavation_Depth_m,Retaining_Wall_Type,Support_System,Deformation_mm,Rainfall_mm_day,Temperature_C,Groundwater_Level_m,Seismic_Activity,Ground_Settlement_mm,Wall_Displacement_mm,Pore_Water_Pressure_kPa,Strain_Gauge,Risk_Level
0,Silt,22.454043,179.770446,513.759461,10.736028,Sheet Pile,Bracing,3.503435,95.429275,2.645510,6.546212,1,70.927201,47.241773,357.894352,50.009234,2
1,Rock,18.402409,169.795469,482.263897,24.671288,Sheet Pile,NaN,2.831278,26.176228,18.377023,7.093143,0,53.211113,38.980432,146.994478,10.330048,2
2,Clay,12.738190,56.410516,386.764476,29.925423,Diaphragm Wall,Anchors,6.018907,47.113187,3.942156,5.861824,0,47.787708,30.105092,238.844635,19.764778,2
3,Silt,25.344875,135.311957,578.023572,3.810702,Sheet Pile,Bracing,29.038429,186.500791,41.729552,3.572420,0,22.349431,22.748016,134.678591,47.963196,1
4,Silt,22.118279,145.048905,200.237258,27.228878,Diaphragm Wall,Bracing,46.438171,171.900595,39.826918,5.770533,1,70.559579,17.808927,53.737179,88.758899,2


## 7. Create Virtual Mine Zones

In [8]:
zone_df = pd.DataFrame({
    "zone_id": ["Z1", "Z2", "Z3"],
    "soil_type": ["rock", "mixed", "clay"],
    "excavation_depth": [20, 50, 80],
    "stress_index": [0.3, 0.6, 0.9]
})

zone_df

,zone_id,soil_type,excavation_depth,stress_index
0,Z1,rock,20,0.3
1,Z2,mixed,50,0.6
2,Z3,clay,80,0.9


## 8. Fuse All Features into Final Dataset

### Derive Base Geotechnical Risk (Engineering Rule-Based)

In [11]:
def derive_base_geotech_risk(stress_index):
    if stress_index < 0.4:
        return 0   # low
    elif stress_index < 0.7:
        return 1   # medium
    else:
        return 2   # high

In [12]:
final_rows = []
timestamp = pd.Timestamp.utcnow()

for _, zone in zone_df.iterrows():
    row = {
        "timestamp": timestamp,
        "zone_id": zone["zone_id"],
        "vibration_energy": seismic_df.iloc[0]["vibration_energy"],
        "vibration_entropy": seismic_df.iloc[0]["vibration_entropy"],
        "acoustic_energy": ae_features.iloc[0]["acoustic_energy"],
        "acoustic_hit_rate": ae_features.iloc[0]["acoustic_hit_rate"],
        "stress_index": zone["stress_index"],
        "soil_type": zone["soil_type"],
        "excavation_depth": zone["excavation_depth"],
        "base_geotech_risk": derive_base_geotech_risk(zone["stress_index"]),
        "risk_label": int(zone["stress_index"] > 0.7)
    }
    final_rows.append(row)

final_df = pd.DataFrame(final_rows)
final_df

,timestamp,zone_id,vibration_energy,vibration_entropy,acoustic_energy,acoustic_hit_rate,stress_index,soil_type,excavation_depth,base_geotech_risk,risk_label
0,2026-01-20 18:37:23.996809+00:00,Z1,1266.302159,"[12.622751715188262, 12.816217014764439, 12.89...",59.408371,10.0,0.3,rock,20,0,0
1,2026-01-20 18:37:23.996809+00:00,Z2,1266.302159,"[12.622751715188262, 12.816217014764439, 12.89...",59.408371,10.0,0.6,mixed,50,1,0
2,2026-01-20 18:37:23.996809+00:00,Z3,1266.302159,"[12.622751715188262, 12.816217014764439, 12.89...",59.408371,10.0,0.9,clay,80,2,1


## 9. Save Final Dataset

In [13]:
final_df.to_csv("../data/processed/final_mine_data.csv", index=False)
print("final_mine_data.csv saved successfully")

final_mine_data.csv saved successfully


In [14]:
final_df.columns

Index(['timestamp', 'zone_id', 'vibration_energy', 'vibration_entropy',
       'acoustic_energy', 'acoustic_hit_rate', 'stress_index', 'soil_type',
       'excavation_depth', 'base_geotech_risk', 'risk_label'],
      dtype='object')

In [15]:
# Number of time steps to simulate
N_TIMESTEPS = 300

# Generate time index (1-minute interval)
time_index = pd.date_range(
    start=pd.Timestamp.utcnow() - pd.Timedelta(minutes=N_TIMESTEPS),
    periods=N_TIMESTEPS,
    freq="1min"
)

In [16]:
zone_profiles = {
    "Z1": {"stress_base": 0.3, "noise": 0.02},
    "Z2": {"stress_base": 0.6, "noise": 0.05},
    "Z3": {"stress_base": 0.9, "noise": 0.08},
}

In [17]:
time_series_rows = []

for zone_id, profile in zone_profiles.items():
    for t in time_index:
        stress = np.clip(
            profile["stress_base"] + np.random.normal(0, profile["noise"]),
            0, 1
        )

        row = {
            "timestamp": t,
            "zone_id": zone_id,
            "vibration_energy": 1200 + stress * 400 + np.random.randn() * 20,
            "vibration_entropy": 10 + stress * 5 + np.random.randn() * 0.3,
            "acoustic_energy": 40 + stress * 30 + np.random.randn() * 2,
            "acoustic_hit_rate": 5 + stress * 10,
            "stress_index": stress,
            "soil_type": "rock" if zone_id == "Z1" else "mixed" if zone_id == "Z2" else "clay",
            "excavation_depth": 20 if zone_id == "Z1" else 50 if zone_id == "Z2" else 80,
            "base_geotech_risk": derive_base_geotech_risk(stress),
            "risk_label": int(stress > 0.7)
        }

        time_series_rows.append(row)

final_df = pd.DataFrame(time_series_rows)
final_df.head()

,timestamp,zone_id,vibration_energy,vibration_entropy,acoustic_energy,acoustic_hit_rate,stress_index,soil_type,excavation_depth,base_geotech_risk,risk_label
0,2026-01-20 13:40:16.007060+00:00,Z1,1294.493529,11.332929,51.749198,7.650625,0.265062,rock,20,0,0
1,2026-01-20 13:41:16.007060+00:00,Z1,1285.716775,11.186969,47.257973,7.714196,0.271420,rock,20,0,0
2,2026-01-20 13:42:16.007060+00:00,Z1,1330.537318,11.805865,50.507939,8.034114,0.303411,rock,20,0,0
3,2026-01-20 13:43:16.007060+00:00,Z1,1330.544053,11.690164,51.677189,7.970798,0.297080,rock,20,0,0
4,2026-01-20 13:44:16.007060+00:00,Z1,1316.414710,11.197017,46.731825,7.727588,0.272759,rock,20,0,0


In [18]:
final_df.to_csv("../data/processed/final_mine_data.csv", index=False)
print("✅ Time-series final_mine_data.csv saved")

✅ Time-series final_mine_data.csv saved


In [19]:
final_df.groupby("zone_id")["risk_label"].mean()

zone_id
Z1    0.00
Z2    0.03
Z3    0.99
Name: risk_label, dtype: float64